<h1> Example to load SC data from InfluxDB in the DOberman v6 schema </h1>

To load data from influx with this script, copy this notebook to your hydra account (`/data/workspace`). Then, you can run it in the jupyterhub in your browser (`http://10.4.73.172:8001/`).

In [ ]:
import os
import requests
from pymongo import MongoClient
import numpy as np
import re

In [ ]:
# First, we connect to MongoDB to collect the connection details for influx db
# If you don't have the mongo_uri and experiment_name saved as environment variable replace with the correct
# Strings in the following two lines.
mongo_uri = os.environ.get('DOBERMAN_MONGO_URI')
experiment_name = os.environ.get('DOBERMAN_EXPERIMENT_NAME')
mongo_client = MongoClient(mongo_uri)
mongo_db = mongo_client[experiment_name]
collection = mongo_db['experiment_config']
influx_cfg = collection.find_one({'name': 'influx'})

# Now we build our influx query url
url = influx_cfg['url'] + '/query?'
headers = {'Accept': 'application/csv',
           'Authorization': f'Token {influx_cfg["token"]}'}
params = {'db': 'slowdata', 'org': experiment_name}

In [ ]:
def get_data(topic, sensor, start='now()-1h', end=None, interval=None):
    """"
    :param topic:    topic of the measurement (e.g. 'temperature')
    :param sensor:   name of the sensor (e.g. 'T_XS_01')
    :param start:    start of query window. possible formats: realtive (e.g. 'now()-10m', 'now()-5d'),
                     timestamp in ns (e.g. '1656925200000000000'),
                     datetime string 'YYYY-MM-DDTHH:MM:SSZ' (e.g. '2022-07-04T09:43:00Z')
    :param end:      end of query window. Same formats as start.
    :param interval: time interval in which data points will be averaged.
                     Use reasonably large interval when querying long time ranges unless you really want 
                     10ks of single measurements(e.g. '10m', '2h', '1d')
    :returns :       (numpy array of timestamps in ns, numpy array of values) 
                  """
    datetime_format = r"(\d{4})-(\d{2})-(\d{2})T(\d{2}):(\d{2}):(\d{2}Z)"
    if re.match(datetime_format, start):
        start = f"'{start}'" # we need additional quotes if we use a datatime string
    condition = f"FROM {topic} WHERE sensor='{sensor}' AND time >= {start}"
    if end:
        if re.match(datetime_format, end):
            end = f"'{end}'" # same here
        condition += f" AND time <= {end}"
    if interval:
        #this averages values per given time interval
        query = f"SELECT mean(value) {condition} GROUP BY time({interval}) fill(none);" 
    else:
        #this get every datapoint between start and end
        query = f"SELECT value {condition};"
    print(query)
    params['q'] = query
    r = requests.get(url, headers=headers, params=params)
    data_points = r.content.decode().splitlines()
    data_array = np.array([d.split(',')[-2:] for d in data_points[1:]])
    if len(data_array):
        timestamps = [int(d) for d in data_array[:,0]]
        values = [float(d) for d in data_array[:,1]]
        print(f'Found {len(values)} data points.')
    else:
        print('ERROR: No data in selection')
        return [], []
    return timestamps, values

In [ ]:
t1,v1 = get_data('temperature', 'T_TS_01', '2022-03-20T00:00:00Z', '2022-03-25T23:00:00Z', '10m')

In [ ]:
t2,v2 = get_data('temperature', 'T_TS_02', '2022-03-20T00:00:00Z', '2022-03-25T23:00:00Z', '10m')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()

plt.scatter(t1, v1, label='bottom')
plt.scatter(t2, v2, label='top')

plt.xlabel('timestamp / ns')
plt.ylabel('T /K')
plt.legend()
plt.title('Thermosyphon temperature')
plt.show()